将不同的数据源进行合并，既包括将两个不同的数据集非常简单地拼接在一起，也包括用数据库那样的连接（join）与合并（merge）操作处理有重叠字段的数据集。Series 与 DataFrame 都具备这类操作，Pandas 的函数与方法让数据合并变得快速简单。

先用 pd.concat 函数演示一个 Series 与 DataFrame 的简单合并操作。
在介绍 Pandas 中更复杂的 merge 和 join 内存数据合并操作。

In [1]:
import numpy as np
import pandas as pd

In [2]:
# 定义一个能够创建 DataFrame 某种形式的函数

def make_df(cols, ind):
    '''一个简单的 DataFrame'''
    data = {c:[str(c) + str(i) for i in ind] for c in cols}
    return pd.DataFrame(data, ind)

# DataFrame 示例
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


# 1-知识回顾：NumPy 数组的合并
    合并 Series 与 DataFrame 与合并 NumPy 数组基本相同，
    NumPy 数组通过 np.concatenate 函数即可完成。

In [3]:
# 可用 np.concatenate 函数将两个或两个以上的数组合并成一个数组。
x = [1,2,3]
y = [4,5,6]
z = [7,8,9]
np.concatenate([x,y,z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [4]:
# 第一个参数是需要合并的数组列表或元组。
# axis 参数可以设 置合并的坐标轴方向
x = [[1,2],[3,4]]
np.concatenate([x,x],axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

# 2-通过 pd.concat 实现简易合并
    Pandas 有一个 pd.concat() 函数与 np.concatenate 语法类似。

pd.concat 的参数更多，功能更强大。

    pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True)

In [5]:
# pd.concat() 可以简单地合并一维的 Series 或 DataFrame 对象，
# 与 np.concatenate() 合并数组一样
ser1 = pd.Series(['A','B','C'], index=[1,2,3])
ser2 = pd.Series(['D','E','F'], index=[4,5,6])
pd.concat([ser1,ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

In [6]:
# 也可以用来合并高维数据
df1 = make_df('AB', [1,2])
df2 = make_df('AB', [3,4])
print(df1);print(df2);
print(pd.concat([df1,df2]))

    A   B
1  A1  B1
2  A2  B2
    A   B
3  A3  B3
4  A4  B4
    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


默认情况下，DataFrame 的合并都是逐行进行的（默认设置是 axis=0）

In [7]:
# 与 np.concatenate() 一样，pd.concat 也可以设置合并坐标轴。
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
print(df3);print(df4);
print(pd.concat([df3, df4], axis=1))

    A   B
0  A0  B0
1  A1  B1
    C   D
0  C0  D0
1  C1  D1
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1


01--索引重复
    
    np.concatenate 与 pd.concat 最主要的差异之一就是 Pandas 在合并时会保留索引，即使索引是重复的

In [8]:
x = make_df('AB',[0,1])
y = make_df('AB',[2,3])
y.index = x.index # 复制索引
print(x); print(y);
print(pd.concat([x, y]))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3


结果中的索引是重复的。

虽然 DataFrame 允许这么做， 但结果并不是我们想要的。pd.concat() 提供了一些解决这个问题的方法。

001-捕捉索引重复的错误
    
    检测 pd.concat() 合并的结果中是否出现了重复的索引，可设置 verify_integrity 参数。
    将参数设置为 True，合并时若有索引重复就会触发异常。

In [9]:
try:
    pd.concat([x, y],verify_integrity=True)
except ValueError as e:
    print('ValueError：',e)

ValueError： Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


002-忽略索引

    有时索引无关紧要，那么合并时就可以忽略它们，可以通过设置 ignore_index 参数来实现。

In [10]:
# 将参数设置为 True，合并时将会创建一个新的整数索引
print(x); print(y);
print(pd.concat([x, y], ignore_index=True))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3


003-增加多级索引

    通过 keys 参数 为数据源设置多级索引标签，这样结果数据就会带上多级索引。

In [11]:
print(x); print(y); 
print(pd.concat([x, y], keys=['x', 'y']))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3


合并后的结果是多级索引的 DataFrame

02-类似 join 的合并
    
    前面介绍的简单示例都有一个共同特点，那就是合并的 DataFrame 都是同样的列名。
    在实际工作中，需要合并的数据往往带有不同的列名，而 pd.concat 提供了一些选项来解决这类合并问题。

In [12]:
# 两个 DataFrame，它们的列名部分相同，却又不完全相同
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
print(df5); print(df6); 
print(pd.concat([df5, df6],sort=False))  # sort=False 接受未来的行为

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4


默认情况下，某个位置上缺失的数据会用 NaN 表示.  如果不想这样，可以用 join 和 join_axes 参数设置合并方式。

默认的合并方式是对所有输入列进行并集合并（join='outer'）,当然也可以用 join='inner' 实现对输入列的交集合并。

In [13]:
print(df5); print(df6);
print(pd.concat([df5, df6], join='inner'))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4


03-append()方法
    
    Series 和 DataFrame 对象都支持 append 方法

In [14]:
print(df1); print(df2);
print(df1.append(df2))

    A   B
1  A1  B1
2  A2  B2
    A   B
3  A3  B3
4  A4  B4
    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


需要注意的是，与 Python 列表中的 append() 和 extend() 方法不同，Pandas 的 append() 不直接更新原有对象的值，而是为合并后的数据创建一个新对象。因此，它不能被称之为一个非常高效的解决方案，因为每次合并都需要重新创建索引和数据缓存。总之，如果你需要进行多个 append 操作，还是建议先创建一个 DataFrame 列表，然后用 concat() 函数一次性解决所有合并任务。